<a href="https://colab.research.google.com/github/abdyraman/harassmentsurvey/blob/main/HarassmentSurvey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#Authentification to google collab
from google.colab import auth
auth.authenticate_user()
#Other libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [45]:
import gspread
from google.auth import default
creds,_=default()

gs=gspread.authorize(creds)
#Get google sheet
# Open the Google Sheet by name (do not include the .gsheet extension)
sheet = gs.open("Harassment in the Workplace (Responses)") # Correct name without .gsheet

# Access the specific worksheets
df_text = pd.DataFrame(sheet.worksheet("Form Responses 1").get_all_records())

In [46]:
#Checking the unique answer points per feature
unique_counts_objects = df_text.select_dtypes('object').nunique()

# Looping through each categorical variable and printing its unique values and counts
for i in unique_counts_objects.index:
    unique_values = df_text[i].value_counts()
    print(f'Unique values of {i}:')
    print(unique_values)
    print()

Unique values of Timestamp:
Timestamp
10/14/2024 11:54:38    1
12/17/2024 11:54:38    1
12/27/2024 11:54:38    1
12/26/2024 11:54:38    1
12/25/2024 11:54:38    1
                      ..
11/14/2024 11:54:38    1
11/13/2024 11:54:38    1
11/12/2024 11:54:38    1
11/11/2024 11:54:38    1
1/22/2025 11:54:38     1
Name: count, Length: 101, dtype: int64

Unique values of Have you received any employer-sponsored training on how to handle potentially violent or harassment situations?:
Have you received any employer-sponsored training on how to handle potentially violent or harassment situations?
Other    42
Yes      34
No       25
Name: count, dtype: int64

Unique values of If yes, has your training prepared you to deal with violent situations that may arise in your working environment?:
If yes, has your training prepared you to deal with violent situations that may arise in your working environment?
I don't know    39
No              34
Yes             28
Name: count, dtype: int64

Unique v

In [47]:
def clean_column_names(columns):
    # Define the patterns to remove
    patterns = [
        r"Please rate the following statements on a scale from 'Strongly Disagree' to 'Strongly Agree' \s*\[",
        r"Please rate the following statements on a scale from 'Never' to 'All of the time'. \s*\[",
    ]
    # Join patterns with `|` for regex
    combined_pattern = '|'.join(patterns)
    # Perform the replacement and ensure spaces are handled correctly
    cleaned_columns = columns.str.replace(combined_pattern, "", regex=True).str.strip()
    # Remove any leading `[` and trailing `]`
    cleaned_columns = cleaned_columns.str.replace(r'^\s*\[', '', regex=True).str.replace(r'\s*\]', '', regex=True)
    # Normalize spaces to a single space, only if needed
    cleaned_columns = cleaned_columns.str.replace(r'\s+', ' ', regex=True)
    return cleaned_columns

In [48]:
# Select object columns from df_text
text = df_text.select_dtypes(include=['object'])
# # Clean the column names
text.columns = clean_column_names(text.columns)

In [49]:
text.columns

Index(['Timestamp',
       'Have you received any employer-sponsored training on how to handle potentially violent or harassment situations?',
       'If yes, has your training prepared you to deal with violent situations that may arise in your working environment?',
       'Have you been trained to recognize and address discrimination, stereotypes, domestic violence and other aspects of human rights-related harassment and violence?',
       'Is there a violence and harassment prevention program at your workplace?',
       'Does your workplace have a written policy concerning violence and harassment?',
       'The organization's policies and processes with respect to prohibiting and reporting inappropriate gender-related behavior are easy to understand and follow',
       'Star performers are held to the same standards as other employees with respect to inappropriate gender-related behavior',
       'I would be comfortable reporting inappropriate conduct by a supervisor',
       'I wou

In [50]:
behavior_interactions=text[["The organization's policies and processes with respect to prohibiting and reporting inappropriate gender-related behavior are easy to understand and follow"
,"Star performers are held to the same standards as other employees with respect to inappropriate gender-related behavior"
,"I would be comfortable reporting inappropriate conduct by a supervisor"
,"I would be comfortable reporting inappropriate sexual conduct by a coworker"
,"There are adverse consequences for senior leaders who are abusive, disrespectful, or hostile"
,"In case I experience or witness harassment in the work environment, I have an action plan on how to report"
,"I am confident that complaints of harassment will be taken seriously and investigated thoroughly"
,"There are adequate confidential channels for reporting harassment"
,"The current measures and procedures for preventing and addressing harassment are effective"
,"The organization's culture promotes respect and safety for all employees"
,"Senior leadership demonstrates a commitment to preventing and addressing harassment"
,"The training programs address the nuances of different types of harassment and discrimination"]]

# Melt the DataFrame to long format
melted_values = behavior_interactions.melt(var_name='Statement', value_name='Rating')

# Count occurrences of each rating per statement
rating_counts_behavior_interactions = melted_values.groupby(['Statement','Rating']).size().reset_index(name='Count')

In [51]:
biases=text[["I believe my opportunities for career success are negatively affected by my gender"
,"I am aware of unpleasant and negative gossip in the workplace"
,"I am aware of abusive, disrespectful, or hostile treatment of employees"
,"I am aware of bullying behavior in the workplace"
,"I have been criticized for my personal communication style or appearance"
,"I have experienced or witnessed unwanted physical conduct in the workplace or by coworkers away from the workplace"
,"I have witnessed or heard of offensive or inappropriate jokes or comments in our workplace"
,"I am aware of sexually inappropriate phone calls, text messages, or social media attention from a coworker"
,"I have been the witness of conduct that I consider to be sexual harassment"
,"I feel unsafe at work because of inappropriate sexual conduct by some individuals"
,"I've seen career opportunities be favorably allocated on the basis of existing or expected sexual interactions"
,"I have considered leaving my job because of inappropriate gender-related behavior in the workplace"
,"I have experienced or witnessed inappropriate gender-related behavior by third parties (such as customers, vendors, and suppliers) associated with our organization"
,"Managers here tolerate or turn a blind eye to inappropriate sexual conduct"
,"I am aware of diminishing language toward colleagues based on their gender, disability and.or ethnic background"
,"I am aware of inappropriate and rude questions and comments regarding the pregnancy of my colleague.s"
,"I am aware of the inappropriate and rude questions and comments regarding the ethnic background of my colleague.s"
,"I am aware of the inappropriate and rude questions and comments regarding the disability of colleague.s"]]

# Melt the DataFrame to long format
melted_values = biases.melt(var_name='Statement', value_name='Rating')

# Count occurrences of each rating per statement
rating_counts_biases= melted_values.groupby(['Statement','Rating']).size().reset_index(name='Count')

# Numeric database creation

In [52]:
# Define the mapping dictionaries
frequency_mapping = {
    'All of the time': 1,
    'Often': 2,
    'Rarely': 3,
    'Sometimes': 4,
    'Never': 5,
}

agreement_mapping = {
    'Strongly Agree': 5,
    'Agree': 4,
    'Neutral': 3,
    'Disagree': 2,
    'Strongly Disagree': 1,
}

# Create a copy of the text DataFrame
df = text.copy()

# Decode the responses
df_biased = df.applymap(lambda x: frequency_mapping.get(x, x))  # Decode using mapping
df_behaviors=df_biased .applymap(lambda x: agreement_mapping.get(x, x))  # Decode using mapping
full_numeric_df=df_behaviors.copy()

<ipython-input-52-abd30e4987e3>:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_biased = df.applymap(lambda x: frequency_mapping.get(x, x))  # Decode using mapping
<ipython-input-52-abd30e4987e3>:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_behaviors=df_biased .applymap(lambda x: agreement_mapping.get(x, x))  # Decode using mapping


In [64]:
stat_numbers=pd.DataFrame(full_numeric_df.describe())
stat_numbers_T=stat_numbers.T.round(2)


**Demographics**

In [55]:
# Group by the specified columns and calculate mean
def dataframing(df, column):
    # Group by the specified column and calculate the mean
    grouped_df = df.groupby([column]).mean(numeric_only=True).reset_index()
    # Round the numeric columns to 2 decimal places
    grouped_df = grouped_df.round(2)
    # Set the specified column as the index
    grouped_df.set_index(column, inplace=True)
    return grouped_df

column= "Which of the following describes your gender?"
gender_df=dataframing(full_numeric_df,column)


In [56]:
# Group by the specified columns and calculate mean and median for a numeric column
column= "Do you identify as a person with a disability?"
disability= dataframing(full_numeric_df,column)
# Rename the "Yes" column
disability_df = disability.rename(index={"Yes": "Identify as a person with a disability"})

def drop_index_from_df(df):
    index_to_drop = ["No", "I prefer not to say"]
    return df.drop(index=index_to_drop)

# # Drop columns and update the DataFrame
disability_df = drop_index_from_df(disability_df)

In [57]:
# Group by the specified columns and calculate mean and median for a numeric column
column= "Do you identify as Indigenous?"
indigenous= dataframing(full_numeric_df,column)
indigenous_df=indigenous.rename(index={"Yes": "Identify as indigenous"})
indigenous_df=drop_index_from_df(indigenous_df)

In [58]:
# Group by the specified columns and calculate mean and median for a numeric column
column= "Do you identify as visible minority?"
minority= dataframing(full_numeric_df,column)
minority_df=minority.rename(index={"Yes": "Identify as visible minority"})
minority_df=drop_index_from_df(minority_df)


In [59]:
#Remove rows that brings no value "No" and "Prefer not to answer" to the df
column= "Are you a member of the 2SLGBTQI+ communities?"
LGBTQ= dataframing(full_numeric_df,column)
LGBTQ_df=LGBTQ.rename(index={"Yes": "A member of the 2SLGBTQI+ communities"})
LGBTQ_df=drop_index_from_df(LGBTQ_df)


In [60]:
#Remove rows that brings no value "No" and "Prefer not to answer" to the df
column= "What industry are you working in?"
industry_df= dataframing(full_numeric_df,column)

In [61]:
#Round and leave only 2 decimals
def transpose_and_add_index(df,decimals=2):
    transposed_df = df.T.round(decimals)
    return transposed_df

# Usage for each DataFrame
gender_df_T= transpose_and_add_index(gender_df)
LGBTQ_df_T = transpose_and_add_index(LGBTQ_df)
industry_df_T = transpose_and_add_index(industry_df)
minority_df_T = transpose_and_add_index(minority_df)
indigenous_df_T = transpose_and_add_index(indigenous_df)
disability_df_T = transpose_and_add_index(disability_df)

In [62]:
# Combine all DataFrames using pd.concat for a horizontal join
dataframes = [
    stat_numbers_T,
    gender_df_T,
    LGBTQ_df_T,
    industry_df_T,
    minority_df_T,
    indigenous_df_T,
    disability_df_T
]

# Perform outer join on indices
statistical_df = pd.concat(dataframes, axis=1, join='outer')
# Reset the index to make it a column
statistical_df.reset_index(inplace=True)
# Rename the new column to "Original Index"
statistical_df.rename(columns={'index': 'Statements'}, inplace=True)


In [63]:
def new_worksheet(variable, title):
    # Set the title for the new worksheet
    new_sheet_title = title
    # Create a new worksheet with the desired title
    worksheet = sheet.add_worksheet(title=new_sheet_title, rows="120", cols="50")  # Adjust rows and cols as needed
    # Update the new worksheet with DataFrame content
    worksheet.update([variable.columns.values.tolist()] + variable.values.tolist())
    return worksheet  # Optional: Return the worksheet object if needed

# Example usage
biases = new_worksheet(rating_counts_biases, "HarassmentBiases")
behavior_interactions=new_worksheet(rating_counts_behavior_interactions, "BehaviorInteractions")
numeric=new_worksheet(full_numeric_df, "Numeric")
statistical_df=new_worksheet(statistical_df, "Demographics")